In [159]:
# Import 
import pandas as pd
import numpy as np
from scipy import signal

In [160]:
# Input file
folder_path = r'D:\iriwa\デスクトップ\Code\Project_Python\Research'
file_name = r'\yoshiyama.csv'
output_filename = '吉山'+'(perclos)'

input_data = pd.read_csv(folder_path + file_name, header=None, skiprows=1,sep=';',names=('No.','timestamp','eyelids_gap','pupil_x','pupil_y','pupil_diameter'))

input_data = input_data.dropna()
print(input_data)

No.                timestamp  eyelids_gap  pupil_x  pupil_y  \
0            1  24-11-2020 14:51:09.687         85.0    224.0    116.0   
1            2  24-11-2020 14:51:09.696         82.0    221.0    118.0   
2            3  24-11-2020 14:51:09.704         83.0    226.0    114.0   
3            4  24-11-2020 14:51:09.712         84.0    219.0    117.0   
4            5  24-11-2020 14:51:09.721         85.0    220.0    118.0   
...        ...                      ...          ...      ...      ...   
221784  221785  24-11-2020 15:21:55.334         41.0    209.0    106.0   
221785  221786  24-11-2020 15:21:55.343         43.0    209.0    106.0   
221786  221787  24-11-2020 15:21:55.352         41.0    209.0    107.0   
221787  221788  24-11-2020 15:21:55.359         44.0    209.0    107.0   
221788  221789  24-11-2020 15:21:55.368         41.0    209.0    107.0   

        pupil_diameter  
0                 37.0  
1                 37.0  
2                 36.0  
3                 39.0

In [161]:
N=128
fs = 120
dt = 1/fs
n = int(len(input_data))
time1 = pd.DataFrame({'time': [i*dt for i in range(n)]})
TIME = int(n/fs)
time2 = pd.DataFrame({'time': [i for i in range(TIME)]})
print(TIME)

1848


In [162]:
df = pd.DataFrame({'gap': [i for i in input_data['eyelids_gap']]})
df = df.interpolate()
max_gap = df['gap'].max()
min_gap = df['gap'].min()

df1 = pd.DataFrame({'perclos': [i/max_gap for i in df['gap']]})
print(df1)


perclos
0       0.841584
1       0.811881
2       0.821782
3       0.831683
4       0.841584
...          ...
221784  0.405941
221785  0.425743
221786  0.405941
221787  0.435644
221788  0.405941

[221789 rows x 1 columns]


In [163]:
p_50 = pd.DataFrame({'perclos50': [1 if i<0.5 else 0 for i in df1['perclos']]})
p_60 = pd.DataFrame({'perclos60': [1 if i<0.4 else 0 for i in df1['perclos']]})
p_70 = pd.DataFrame({'perclos70': [1 if i<0.3 else 0 for i in df1['perclos']]})
p_80 = pd.DataFrame({'perclos80': [1 if i<0.2 else 0 for i in df1['perclos']]})
p_90 = pd.DataFrame({'perclos90': [1 if i<0.1 else 0 for i in df1['perclos']]})


In [164]:
per_50 = pd.DataFrame({'perclos50': [p_50.iloc[i*fs:i*fs+fs,0].sum() for i in range(TIME)]})
per_60 = pd.DataFrame({'perclos60': [p_60.iloc[i*fs:i*fs+fs,0].sum() for i in range(TIME)]})
per_70 = pd.DataFrame({'perclos70': [p_70.iloc[i*fs:i*fs+fs,0].sum() for i in range(TIME)]})
per_80 = pd.DataFrame({'perclos80': [p_80.iloc[i*fs:i*fs+fs,0].sum() for i in range(TIME)]})
per_90 = pd.DataFrame({'perclos90': [p_90.iloc[i*fs:i*fs+fs,0].sum() for i in range(TIME)]})

perclos_50 = per_50.rolling(60).sum()/7200
perclos_60 = per_60.rolling(60).sum()/7200
perclos_70 = per_70.rolling(60).sum()/7200
perclos_80 = per_80.rolling(60).sum()/7200
perclos_90 = per_90.rolling(60).sum()/7200

In [165]:
output_data = pd.concat([time2, perclos_50, perclos_60, perclos_70, perclos_80, perclos_90], axis=1)
print(output_data)


time  perclos50  perclos60  perclos70  perclos80  perclos90
0        0        NaN        NaN        NaN        NaN        NaN
1        1        NaN        NaN        NaN        NaN        NaN
2        2        NaN        NaN        NaN        NaN        NaN
3        3        NaN        NaN        NaN        NaN        NaN
4        4        NaN        NaN        NaN        NaN        NaN
...    ...        ...        ...        ...        ...        ...
1843  1843   0.207639   0.170278   0.129306   0.071528   0.038611
1844  1844   0.224306   0.186944   0.135694   0.072222   0.038611
1845  1845   0.240972   0.203611   0.142778   0.074028   0.039583
1846  1846   0.257500   0.214028   0.142778   0.074028   0.039583
1847  1847   0.274167   0.230556   0.146528   0.076250   0.040972

[1848 rows x 6 columns]


In [166]:
# Output data
output_folder_path = 'D:\\iriwa\\デスクトップ\\Code\\Project_Python\\Research\\perclos'
output_data.to_excel(output_folder_path + '\\' + output_filename +'.xlsx', sheet_name=output_filename, index=False)
